In [8]:
import pandas as pd
import os

In [9]:
# Output file will end up in the path below
path_stem = "/Users/ruthjohnson/Desktop/CS230/stat_analysis/"

# Directory containing pylint output - download from https://drive.google.com/open?id=1DLiYPP594fUsGQsNAM5lU_b_-xQ2xxvg
pylint_path = path_stem + "Pylint_output/"

# Directory containing bandit output - download from https://drive.google.com/open?id=1lxTNIE0wMLc0Ixb6HhcQxgLReG_jTpA6
bandit_path = path_stem + "Bandit_output/"

# Path for intersection analysis
intersection_path = path_stem + "Intersection/"

# I. Build PyLint Dataframe

In [10]:
directory = os.fsencode(pylint_path) 
dic = dict()

for file in os.listdir(directory): 
    filename = os.fsdecode(file) 
    benchmark_name = filename.split('_pylint.txt')[0]
    full_path = pylint_path + filename
    df = pd.read_csv(full_path)
    dic[benchmark_name] = df.sum()
    
pylint_df = pd.DataFrame.from_dict(dic).T
pylint_df['TOTAL_PYLINT'] = pylint_df.sum(axis=1)

In [11]:
len(pylint_df)

650

In [12]:
pylint_df.head()

,I,R,C,W,E,F,TOTAL_PYLINT
Radicale-1.1.1,0.0,39.0,56.0,69.0,78.0,0.0,242.0
trio-websockets-0.2,0.0,0.0,44.0,9.0,41.0,0.0,94.0
zhmcclient-0.20.0,0.0,556.0,1521.0,1071.0,116.0,0.0,3264.0
ddtrace-0.10.1,0.0,55.0,600.0,191.0,69.0,0.0,915.0
passlib-1.3.1_pylint(3).txt,0.0,66.0,1794.0,573.0,86.0,0.0,2519.0


# II. Build Bandit DataFrame

In [13]:
# List of bandit tests from https://github.com/PyCQA/bandit
test_ids = ["B101", "B102", "B103", "B104", "B105", "B106", "B107", "B108", "B109", "B110", "B111", "B112"]
test_ids += ["B201"]
test_ids += ["B301", "B302", "B303", "B304", "B305", "B306", "B307", "B308", "B309", "B310"]
test_ids += ["B311", "B312", "B313", "B314", "B315", "B316", "B317", "B318", "B319", "B320"]
test_ids += ["B321", "B322", "B323", "B324", "B325"]
test_ids += ["B401", "B402", "B403", "B404", "B405", "B406", "B407", "B408", "B409", "B410", "B411", "B412", "B413"]
test_ids += ["B501", "B502", "B503", "B504", "B505", "B506", "B507"]
test_ids += ["B601", "B602", "B603", "B604", "B605", "B606", "B607", "B608", "B609", "B610", "B611"]
test_ids += ["B701", "B702", "B703"]

In [18]:
directory = os.fsencode(bandit_path) 
dic = dict()

for file in os.listdir(directory): 
    filename = os.fsdecode(file) 
    print(filename)
    benchmark_name = filename.split('_bandit.txt')[0]
    full_path = bandit_path + filename
    try:
        df = pd.read_csv(full_path)
    except:
        print("%s cannot be read" % full_path)
    inner_dic = dict()

    # Filter out the low confidence issues
    df = df[df['issue_confidence'] != 'LOW']
    TOTAL_BANDIT = len(df)
    inner_dic['TOTAL_BANDIT'] = TOTAL_BANDIT

    # Count number of issues in each severity category
    HIGH_SEVERITY = len(df[df['issue_severity'] == 'HIGH'])
    MED_SEVERITY = len(df[df['issue_severity'] == 'MEDIUM'])
    LOW_SEVERITY = len(df[df['issue_severity'] == 'LOW'])
    inner_dic['HIGH_SEVERITY'] = HIGH_SEVERITY
    inner_dic['MED_SEVERITY'] = MED_SEVERITY
    inner_dic['LOW_SEVERITY'] = LOW_SEVERITY

    # Count the number of issues in each test category
    id_sum = 0
    for test_id in test_ids:
        id_count = len(df[df['test_id']==test_id])
        inner_dic[test_id] = id_count
        id_sum += id_count
    if (id_sum != TOTAL_BANDIT):
        print("ERROR: Script didn't catch all Bandit tests in" + benchmark_name)
        print("Number of issues caught: " + id_sum)
        print("Number of issues expected: " + TOTAL_BANDIT)
        print("==================================================================")
    
    # Append to dictionary of files 
    dic[benchmark_name] = inner_dic
    
bandit_df = pd.DataFrame.from_dict(dic).T

restkit-4.2.2_bandit.txt
streamsx.objectstorage-1.1.0_bandit.txt
recurly-2.6.2_bandit.txt
peewee-2.9.2_bandit.txt
drf-tracking-1.2.0_bandit.txt
pconf-1.3.2_bandit.txt
aiocouchdb-0.5.0_bandit.txt
zhmc-ansible-modules-0.5.2_bandit.txt
textract-1.4.0_bandit.txt
mlalchemy-0.2.1_bandit.txt
TracAuthOpenId-0.4.3_bandit.txt
bottle-0.12.9_bandit.txt
web.py-0.38_bandit.txt
django-crispy-forms-1.1.3_bandit.txt
django-fiber-0.9.9_bandit.txt
boss-cli-0.3.0_bandit.txt
astropy-2.0.12_bandit.txt
archmage-0.3_bandit.txt
pysam-0.11.1_bandit.txt
Werkzeug-0.11.10_bandit.txt
urllib3-1.22_bandit.txt
django-uni-form-0.8.0_bandit.txt
django-ca-1.9.0_bandit.txt
ansible-vault-1.0.4_bandit.txt
suds-0.4_bandit.txt
mollie-api-python-2.0.3_bandit.txt
anncolvar-0.3_bandit.txt
pypicloud-0.2.1_bandit.txt
oauthlib-0.6.3_bandit.txt
output_no_docker_bandit.txt
plone.app.users-1.0.4_bandit.txt
coincurve-7.1.0_bandit.txt
Products.CMFQuickInstallerTool-3.0.13_bandit.txt
django-access-tokens-0.9.1_bandit.txt
plone.app.conten

IM-1.4.8_bandit.txt
tweepy-2.3.0_bandit.txt
passlib-1.3.1_bandit.txt
lambda-tools-0.1.1_bandit.txt
foolscap-0.6.5_bandit.txt
django-airplane-0.2.0_bandit.txt
bepasty-0.1.0_bandit.txt
newrelic-2.106.0.87_bandit.txt
annotator-0.10.0_bandit.txt
restview-2.8.0_bandit.txt
seed-stage-based-messaging-0.10.1_bandit.txt
django-storages-1.6.6_bandit.txt
twilio-3.4.5_bandit.txt
requests-2.2.1_bandit.txt
python-nomad-1.1.0_bandit.txt
django-sticky-uploads-0.1.0_bandit.txt
bjoern-1.4.1_bandit.txt
pakettikauppa-0.1.1_bandit(1).txt
virtualenv-1.4.9_bandit.txt
requests-kerberos-0.5_bandit.txt
django-html5-appcache-0.2.2_bandit.txt
splunk-sdk-1.6.5_bandit.txt
Flask-0.12.2_bandit.txt
stargate-0.3_bandit.txt
CLAM-0.9.9.1_bandit.txt
plumi.app-4.1.2_bandit.txt
pyramid-1.5.8_bandit.txt
confire-0.2.0_bandit.txt
gevent-1.1.2_bandit(1).txt
python-keystoneclient-1.3.4_bandit.txt
definitions-0.2.0_bandit.txt
django-hashedfilenamestorage-2.3.1_bandit.txt
UnleashClient-1.0.1_bandit.txt
girder-2.1.1_bandit.txt
djan

In [19]:
len(bandit_df)

492

In [20]:
bandit_df.head()

,B101,B102,B103,B104,B105,B106,B107,B108,B109,B110,...,B609,B610,B611,B701,B702,B703,HIGH_SEVERITY,LOW_SEVERITY,MED_SEVERITY,TOTAL_BANDIT
restkit-4.2.2,30,2,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,1,45,7,53
streamsx.objectstorage-1.1.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
recurly-2.6.2,1,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,9,15,24
peewee-2.9.2,4,1,0,0,4,0,0,0,0,1,...,0,0,0,0,0,0,4,13,8,25
drf-tracking-1.2.0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,7,0,7


# III. Combine Pylint and Bandit Dataframes

In [21]:
combined_df = pylint_df.join(bandit_df).sort_index()
combined_df.head()

,I,R,C,W,E,F,TOTAL_PYLINT,B101,B102,B103,...,B609,B610,B611,B701,B702,B703,HIGH_SEVERITY,LOW_SEVERITY,MED_SEVERITY,TOTAL_BANDIT
AuthBWC-0.1.3,0.0,27.0,628.0,65.0,147.0,0.0,867.0,253.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,258.0,0.0,258.0
AuthBWC-0.1.3_pylint(1).txt,0.0,27.0,628.0,65.0,147.0,0.0,867.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AuthBWC-0.1.3_pylint(2).txt,0.0,27.0,628.0,65.0,147.0,0.0,867.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Beaker-0.9.3,0.0,34.0,443.0,49.0,48.0,0.0,574.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,9.0,5.0,15.0
Beaker-0.9.3_pylint(1).txt,0.0,34.0,443.0,49.0,48.0,0.0,574.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
len(combined_df)

650

# IV. Add a column for file size

In [24]:
benchmark_size = pd.read_csv(path_stem + "benchmark_size.csv", header=None).set_index(0).rename({1:'BENCHMARK_SIZE'}, axis='columns')
benchmark_size.head()

,BENCHMARK_SIZE
0,
aiida-core-0.12.2,9256
aiocouchdb-0.5.0,412
aiohttp-0.16.2,8316
aioli-0.0.4,72
aldryn-django-1.6.11.1,128


In [25]:
combined_df = combined_df.join(benchmark_size)
combined_df.head()

,I,R,C,W,E,F,TOTAL_PYLINT,B101,B102,B103,...,B610,B611,B701,B702,B703,HIGH_SEVERITY,LOW_SEVERITY,MED_SEVERITY,TOTAL_BANDIT,BENCHMARK_SIZE
AuthBWC-0.1.3,0.0,27.0,628.0,65.0,147.0,0.0,867.0,253.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,258.0,0.0,258.0,336.0
AuthBWC-0.1.3_pylint(1).txt,0.0,27.0,628.0,65.0,147.0,0.0,867.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AuthBWC-0.1.3_pylint(2).txt,0.0,27.0,628.0,65.0,147.0,0.0,867.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Beaker-0.9.3,0.0,34.0,443.0,49.0,48.0,0.0,574.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,9.0,5.0,15.0,192.0
Beaker-0.9.3_pylint(1).txt,0.0,34.0,443.0,49.0,48.0,0.0,574.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
len(combined_df)

650

# V. Add Intersections between Bandit and Pylint

In [71]:
# intersect = pd.read_csv(intersection_path)
# intersect.head()

,filename,line_number,pylint_message,pylint_type,pylint_symbol,bandit_issue_text,bandit_issue_confidence,bandit_issue_severity


In [27]:
directory = os.fsencode(intersection_path) 
dic = dict()

for file in os.listdir(directory): 
    filename = os.fsdecode(file) 
    benchmark_name = filename.split('_intersection_intersect_lines.csv')[0]
    full_path = intersection_path + filename
    df = pd.read_csv(full_path)
    inner_dic = dict()

    # Filter out the low confidence issues
    df = df[df['bandit_issue_confidence'] != 'LOW']
    TOTAL_INTERSECTIONS = len(df)
    inner_dic['TOTAL_INTERSECTIONS'] = TOTAL_INTERSECTIONS

    # Count number of issues in each bandit severity category
    inner_dic['HIGH_SEVERITY_INTERSECTIONS'] = len(df[df['bandit_issue_severity'] == 'HIGH'])
    inner_dic['MED_SEVERITY_INTERSECTIONS'] = len(df[df['bandit_issue_severity'] == 'MEDIUM'])
    inner_dic['LOW_SEVERITY_INTERSECTIONS'] = len(df[df['bandit_issue_severity'] == 'LOW'])

    # Count the number of issues in each pylint category
    inner_dic['PYLINT_I_INTERSECTIONS'] = len(df[df['pylint_type'] == 'Category.I'])
    inner_dic['PYLINT_R_INTERSECTIONS'] = len(df[df['pylint_type'] == 'Category.R'])
    inner_dic['PYLINT_C_INTERSECTIONS'] = len(df[df['pylint_type'] == 'Category.C'])
    inner_dic['PYLINT_W_INTERSECTIONS'] = len(df[df['pylint_type'] == 'Category.W'])
    inner_dic['PYLINT_E_INTERSECTIONS'] = len(df[df['pylint_type'] == 'Category.E'])
    inner_dic['PYLINT_F_INTERSECTIONS'] = len(df[df['pylint_type'] == 'Category.F'])
    
    # Append to dictionary of files 
    dic[benchmark_name] = inner_dic
    
intersection_df = pd.DataFrame.from_dict(dic).T

In [28]:
len(intersection_df)

391

In [29]:
intersection_df.head()

,HIGH_SEVERITY_INTERSECTIONS,LOW_SEVERITY_INTERSECTIONS,MED_SEVERITY_INTERSECTIONS,PYLINT_C_INTERSECTIONS,PYLINT_E_INTERSECTIONS,PYLINT_F_INTERSECTIONS,PYLINT_I_INTERSECTIONS,PYLINT_R_INTERSECTIONS,PYLINT_W_INTERSECTIONS,TOTAL_INTERSECTIONS
homeassistant-0.36.1,1,0,1,0,1,0,0,1,0,2
djangocms-highlightjs-0.3.0,0,0,0,0,0,0,0,0,0,0
anncolvar-0.3,0,0,0,0,0,0,0,0,0,0
tryton-2.2.13,1,55,1,48,3,0,0,0,6,57
django-airplane-0.2.0,0,0,0,0,0,0,0,0,0,0


In [30]:
combined_df = combined_df.join(intersection_df)
combined_df.head()

,I,R,C,W,E,F,TOTAL_PYLINT,B101,B102,B103,...,HIGH_SEVERITY_INTERSECTIONS,LOW_SEVERITY_INTERSECTIONS,MED_SEVERITY_INTERSECTIONS,PYLINT_C_INTERSECTIONS,PYLINT_E_INTERSECTIONS,PYLINT_F_INTERSECTIONS,PYLINT_I_INTERSECTIONS,PYLINT_R_INTERSECTIONS,PYLINT_W_INTERSECTIONS,TOTAL_INTERSECTIONS
AuthBWC-0.1.3,0.0,27.0,628.0,65.0,147.0,0.0,867.0,253.0,0.0,0.0,...,0.0,12.0,0.0,10.0,2.0,0.0,0.0,0.0,0.0,12.0
AuthBWC-0.1.3_pylint(1).txt,0.0,27.0,628.0,65.0,147.0,0.0,867.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AuthBWC-0.1.3_pylint(2).txt,0.0,27.0,628.0,65.0,147.0,0.0,867.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Beaker-0.9.3,0.0,34.0,443.0,49.0,48.0,0.0,574.0,1.0,0.0,0.0,...,0.0,7.0,0.0,4.0,2.0,0.0,0.0,0.0,1.0,7.0
Beaker-0.9.3_pylint(1).txt,0.0,34.0,443.0,49.0,48.0,0.0,574.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
len(combined_df)

650

In [32]:
combined_df.columns

Index(['I', 'R', 'C', 'W', 'E', 'F', 'TOTAL_PYLINT', 'B101', 'B102', 'B103',
       'B104', 'B105', 'B106', 'B107', 'B108', 'B109', 'B110', 'B111', 'B112',
       'B201', 'B301', 'B302', 'B303', 'B304', 'B305', 'B306', 'B307', 'B308',
       'B309', 'B310', 'B311', 'B312', 'B313', 'B314', 'B315', 'B316', 'B317',
       'B318', 'B319', 'B320', 'B321', 'B322', 'B323', 'B324', 'B325', 'B401',
       'B402', 'B403', 'B404', 'B405', 'B406', 'B407', 'B408', 'B409', 'B410',
       'B411', 'B412', 'B413', 'B501', 'B502', 'B503', 'B504', 'B505', 'B506',
       'B507', 'B601', 'B602', 'B603', 'B604', 'B605', 'B606', 'B607', 'B608',
       'B609', 'B610', 'B611', 'B701', 'B702', 'B703', 'HIGH_SEVERITY',
       'LOW_SEVERITY', 'MED_SEVERITY', 'TOTAL_BANDIT', 'BENCHMARK_SIZE',
       'HIGH_SEVERITY_INTERSECTIONS', 'LOW_SEVERITY_INTERSECTIONS',
       'MED_SEVERITY_INTERSECTIONS', 'PYLINT_C_INTERSECTIONS',
       'PYLINT_E_INTERSECTIONS', 'PYLINT_F_INTERSECTIONS',
       'PYLINT_I_INTERSECTIONS', 'P

# VI. Export the Combined Dataframe

In [33]:
combined_df.to_csv(path_stem + "combined_df.txt")

In [36]:
combined_df = combined_df.drop_duplicates()

In [37]:
len(combined_df)

563

In [38]:
combined_df.to_csv(path_stem + "combined_df.txt")